In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import nltk
import io
from textblob import Word
import re
import sys, os, csv
import string
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [ ]:
from collections import Counter
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
data_train = pd.read_csv('C:\\Users\\Vishnu Vardhan\\Desktop\\capstone project file\\cleaned_data\\training.csv', sep=',')
print("Dataset shape:",data_train.shape)

In [ ]:
data_train.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

count = data_train.iloc[:,1].value_counts()
fig = plt.figure(figsize=(9,7))  # Rename plt to fig
sns.barplot(x = count.index, y = count.values, alpha=0.8, palette="plasma")
plt.ylabel('Count', fontsize=12)
plt.xlabel('Emotions', fontsize=12)

#sadness (0)
#joy (1)
#love (2)
#anger (3)
#fear (4)
#neutral (5)

In [ ]:
X_train = data_train.iloc[:,0][:14399]
#[:47583]
y_train = data_train.iloc[:,-1][:14399]
#[:47583]
X_val = data_train.iloc[:,0][14400:]
#[47584:]
y_val = data_train.iloc[:,-1][14400:]
#[47584:]

In [ ]:
# Extracting Count Vectors Parameters
count_vect = CountVectorizer(analyzer='word')
count_vect.fit(data_train.iloc[:,0].astype('U'))
X_train_count =  count_vect.transform(X_train.astype('U'))
X_val_count =  count_vect.transform(X_val.astype('U'))
print(count_vect.vocabulary_)

In [ ]:
bow = count_vect.fit_transform(data_train.iloc[:,0].astype('U'))
print(bow.shape)
word_freq = dict(zip(count_vect.get_feature_names(), np.asarray(bow.sum(axis=0)).ravel()))
word_counter = collections.Counter(word_freq)
word_counter_df = pd.DataFrame(word_counter.most_common(30), columns = ['word', 'freq'])
fig, ax = plt.subplots(figsize=(15, 10))
sns.barplot(x="word", y="freq", data= word_counter_df, ax=ax, palette="plasma")
plt.show();

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
logreg1 = LogisticRegression(C=1, max_iter=500)
logreg1.fit(X_train_count, y_train)
y_pred = logreg1.predict(X_val_count)
accuracy = accuracy_score(y_val, y_pred)
conf_matrix_lr = confusion_matrix(y_val, y_pred)
class_report = classification_report(y_val, y_pred)
print("Accuracy:", accuracy)
print("Confusion matrix:\n", conf_matrix_lr)
print("Classification report:\n", class_report)

In [ ]:
import pickle
from flask import Flask,request, jsonify,render_template

In [ ]:
from flask import Flask, redirect, url_for, render_template

In [ ]:
with open('model.pkl', 'wb') as file:
    pickle.dump(logreg1, file)

In [ ]:
model = pickle.load(open("model.pkl", "rb"))

In [ ]:
app = Flask(__name__)

In [ ]:
#app = Flask(__name__, static_url_path='/static')

In [ ]:
@app.route('/')
def home():
    return render_template('home.html')


In [ ]:
#@app.route('/result')
##   return render_template('result.html')

In [ ]:
@app.route('/result', methods=['POST'])
def results():
    return render_template('result.html')

In [ ]:
#def prediction_emotion(tweet):
   # tweet_count = count_vect.transform(tweet)
   # tweet_pred = logreg1.predict(tweet_count)
    # convert predictions to emotion labels
    #emotion_labels = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'neutral'}
    #predicted_emotion = [emotion_labels[pred] for pred in tweet_pred]
    #return predicted_emotion


In [ ]:
#def prediction_emotion(text):
    #count_vect = CountVectorizer()
    #tweets = [text]
    #tweet_count = count_vect.fit_transform(tweets)
   # tweet_pred = model.predict(tweet_count)
    #label_dict = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear',5: 'neutral'}
    #predicted_emotion= label_dict[np.argmax(tweet_pred)]
    #return predicted_emotion*/

In [ ]:
@app.route('/prediction', methods=['POST'])
def prediction():
    return render_template('prediction.html')

In [ ]:
@app.route('/predict', methods=['GET','POST'])
def predict():
    text = request.form.get('text_1')
    tweet_count = count_vect.transform([text])
    tweet_pred = logreg1.predict(tweet_count)
    # convert predictions to emotion labels
    emotion_labels = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'neutral'}
    predicted_emotion = emotion_labels[tweet_pred[0]]
    return render_template('prediction.html', predicted_emotion=predicted_emotion)

In [ ]:
if __name__ == "__main__":
    app.run()
   